In [2]:
# このセルを実行（▶️をクリック）
import subprocess
from pathlib import Path

template_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/template.html")
with template_path.open("r", encoding="utf-8") as f:
    template = f.read()

assignment_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/assignment.txt")
assignment_text = assignment_path.read_text(encoding="utf-8").strip()

input_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/input.txt")
input_text = input_path.read_text(encoding="utf-8")

answer_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/微分積分/answer.txt")
answer_text = answer_path.read_text(encoding="utf-8")

instruction_header = f"""まず、@homework/微分積分/output/solution.htmlの名前を適切に変更し、@homework/微分積分/archive/に移動させた後、以下の情報を参照して解答を作成してください。

【参照ファイル】
- @homework/微分積分/input.txt: 問題が記載されています
- @homework/微分積分/answer.txt: 各問題の解答が記載されています
- @homework/微分積分/assignment.txt: 今回解くべき問題は「{assignment_text}」です

【問題と解答の内容】
--- input.txt（問題） ---
{input_text}
--- input.txt ここまで ---

--- answer.txt（解答） ---
{answer_text}
--- answer.txt ここまで ---

【指示】
`assignment.txt`に記載された問題番号「{assignment_text}」に対応する問題を`input.txt`から見つけ、その問題の解答を`answer.txt`を参照しながら、初学者にも超わかりやすく丁寧に解説してください。

解答を作成する際は：
1. `answer.txt`の解答内容を基に、計算過程を省略せずにすべて詳しく説明します
2. 各ステップで「なぜその計算をするのか」「どの公式を使っているのか」を明確に説明します
3. 数式の変形の理由を必ず文章で補足します
4. 初学者がつまずきやすいポイントには`note`ブロックで補足説明を加えます

出力は次のテンプレートと文体を厳守した HTML を @homework/微分積分/output/solution.html に保存すること。

【テンプレート仕様】
- 下記 HTML をそのまま骨格として使用し、問題タイトル・本文・数式・説明だけを差し替える。
- class 名、タグ階層、CSS を一切変更しない。不要な要素を追加しない。
- MathJax はテンプレートの設定のまま利用し、数式は `$...$`（行内）と `$$...$$`（ディスプレイ）を使用。
"""

instruction_footer = """【執筆ルール】
- `assignment.txt`に記載された問題番号のみを処理する。`h1`は「微分積分 問◯（番号）解答」とし、`h2`は「【解答】」固定で始める。問題文ブロックは`div.problem`内に`【問題】`と対象関数を必ず記載する。
- 問題文ブロック（`div.problem`）と`h2`の「【解答】」の間に、必ず「【方針】」セクションを追加します。このセクションは`div.strategy`で囲み、`div.strategy-title`に「【方針】」というタイトルを入れます。
- 方針セクション内では、まず最初に「必要な知識・テクニック」の紹介セクションを追加します。このセクションは`div.knowledge`で囲み、`div.knowledge-title`に「必要な知識・テクニック」というタイトルを入れます。このセクションでは、問題を解くために必要な公式・定理・テクニックを箇条書きで列挙し、それぞれについて簡潔に特徴や使用場面を説明します（例：「積の微分法：2つの関数の積を微分する際に使用する。$(fg)' = f'g + fg'$の形で表される。」）。このセクションにより、読者が解答を読む前に必要な前提知識を把握できるようにします。
- その後に、`div.flowchart`を使用してフローチャートを作成します。フローチャートは以下の構造で作成します：
  * 各ステップは`div.flowchart-step`で表現し、適切なクラス（`start`：開始、`process`：処理、`decision`：条件分岐、`end`：終了）を付与します
  * ステップ間には`div.flowchart-arrow`で矢印を挿入します
  * 各ステップ内には`div.flowchart-label`でステップ番号やタイトルを表示し、その下にステップの説明を記載します
  * フローチャートは解答全体の流れを視覚的に表現し、以下のような要素を含めます：問題の理解、必要な公式・定理の確認、計算手順、判定基準、結果の検証など
  * 条件分岐がある場合は、判定結果（Yes/No）によって異なる処理が分かれることを示します
  * フローチャートは縦方向に流れるように配置し、各ステップは簡潔に（1〜2行程度）記述します
  * フローチャートのHTML構造例は`template.html`のコメント内に記載されていますので、それを参照してください
- `h2`の「【解答】」の後、最初の`step`ブロックの前に、解答全体の方針を説明する段落を必ず追加します。この段落では、どのような手順・アプローチで問題を解いていくかを簡潔に概説します（例：「この問題では、まず〜を行い、次に〜を求め、最後に〜を確認します。」）。全体の方針を示すことで、読者が解答の流れを事前に把握できるようにします。ただし、この段落はフローチャートで視覚的に表現されている内容を文章で補足する形にします。
- 各`step`ブロックは`answer.txt`の解答を基に、計算過程を省略せずに詳しく説明します。`step-title`は「（1）〜」形式で連番を振ります。段落の語尾は「〜します。」で統一します。各ステップはフローチャートで示された手順に対応するように構成します。
- 数式を示したら必ず直後に文章で解説し、式だけで終わらせません。`formula`ブロックの前後に説明段落を入れます。
- `answer.txt`に記載されている計算過程や判定基準はすべて含め、省略しないでください。
- `note`、`calculation`、`verification`の各ブロックもテンプレートのスタイルで使用し、初学者が理解しやすいように補足説明を加えます。
- 検証セクションには少なくとも2つの確認手順を段落で述べ、最後に「以上より求めた結果の正しさを確認しました。」という文で締めます。
- 末尾に追加のスクリプトやlive-serverのコードは禁止です。

以上を厳守し、`answer.txt`の解答内容を基に、丁寧な微分積分解説を生成してください。"""

command = (
    instruction_header
    + "\n--- HTML テンプレート（そのまま利用） ---\n"
    + template
    + "\n--- テンプレートここまで ---\n\n"
    + instruction_footer
)

try:
    # 1回目のプロンプトを送信
    subprocess.run('pbcopy', text=True, input=command, check=True)
    subprocess.run(['osascript', '-e', '''
    tell application "Cursor"
        activate
        delay 0.3
    end tell
    tell application "System Events"
        keystroke "i" using command down
        delay 0.3
        keystroke "v" using command down
        delay 0.3
        keystroke return
        delay 1.0
    end tell
    '''], check=True)
    print('✅ プロンプト 1/2 を送信しました...')
    
    # 2回目のプロンプトを送信（すでにComposerが開いているので、ペーストとエンターのみ）
    second_prompt = "@solution.html の内容を入念に調べ間違いがないか確認して"
    subprocess.run('pbcopy', text=True, input=second_prompt, check=True)
    subprocess.run(['osascript', '-e', '''
    tell application "Cursor"
        activate
        delay 0.3
    end tell
    tell application "System Events"
        keystroke "v" using command down
        delay 0.3
        keystroke return
    end tell
    '''], check=True)
    print('✅ プロンプト 2/2 を送信しました...')
    print('✅ プロンプトを2回送信完了 - 微分積分問題解説生成中...')
except Exception as e:
    print(f'❌ エラー: {e}')
    print('手動: Cmd+I → 下記を貼り付け')
    print(f'\n1回目:\n{command}')
    print(f'\n2回目:\n@solution.html の内容を入念に調べて読みやすい解答となるように整理して')


✅ プロンプト 1/2 を送信しました...
✅ プロンプト 2/2 を送信しました...
✅ プロンプトを2回送信完了 - 微分積分問題解説生成中...
